In [1]:
# !pip install transformers
# !pip install sentencepiece
# !pip install evaluate rouge_score

In [40]:
import pandas as pd
import os
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import XLNetTokenizer, XLNetLMHeadModel, AdamW
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [41]:
torch.cuda.empty_cache()
import gc
gc.collect()

140

### Read Data

In [42]:
newsela_path = 'newsela/'
newsela_metadata = os.path.join(newsela_path, 'articles_metadata.csv')
newsela_articles_dir = os.path.join(newsela_path, 'articles')

In [43]:
# df = pd.read_csv(newsela_metadata)
df = pd.read_csv('newsela_all.csv')
df.head()

,slug,language,title,grade_level,version,filename,text
0,10dollarbill-woman,en,"Tubman, Perkins or Roosevelt? Woman on $10 bil...",12.0,0,10dollarbill-woman.en.0.txt,WASHINGTON — An abolitionist. The longest-serv...
1,10dollarbill-woman,en,Americans weigh in to choose the woman who wil...,8.0,1,10dollarbill-woman.en.1.txt,WASHINGTON — The all-male lineup on American m...
2,10dollarbill-woman,en,The $10 question: Who will be the new face on ...,6.0,2,10dollarbill-woman.en.2.txt,WASHINGTON — It's time for a woman to be honor...
3,10dollarbill-woman,en,New $10 bill will have a theme and a woman's p...,5.0,3,10dollarbill-woman.en.3.txt,WASHINGTON — It is time that a woman be on Ame...
4,10dollarbill-woman,en,We will soon have an American woman's face on ...,3.0,4,10dollarbill-woman.en.4.txt,WASHINGTON — Pictures of men are on all Americ...


### One-time Processing for text ingestion

In [44]:
# def read_file(file_path):
#     try:
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     except Exception as e:
#         # In case of error, return NaN or some error indication
#         return pd.NA

In [45]:
# texts = []

# for i, row in enumerate(df.itertuples()):
#     file_path = os.path.join(newsela_articles_dir, row.filename)
#     # print(file_path)
#     file_text = read_file(file_path)
#     # print(file_text)
#     texts.append(file_text)

# df['text'] = texts

In [46]:
# df.head()

In [47]:
# df.to_csv('newsela_all.csv', index=False)

### Create Train Test Split

In [48]:
train_index = 8629 # 80% of the entire set
test_index = df.shape[0] - train_index
print(f"Train rows: 0 - {train_index}; Test index: {train_index + 1} - {df.shape[0]}")

Train rows: 0 - 8629; Test index: 8630 - 10786


In [49]:
train_df = df[:train_index]
test_df = df[train_index:]
print(train_df.shape, test_df.shape)

(8629, 7) (2157, 7)


In [50]:
dataset_dict = {"train": [], "test": []}

#### Train

In [51]:
train_dict = train_df.to_dict(orient='records')

In [52]:
prev_slug = train_dict[0]['slug']
tmp_texts = []
tmp_titles = []

for record in train_dict:
    # Iterate through all the records belonging to a slug and save the texts
    if record['slug'] == prev_slug:
        tmp_texts.append(record['text'])
        tmp_titles.append(record['title'])
    # We've reached a new slug, so it's time to create the train/test record for the given slug
    else:
        for title, text in zip(tmp_titles[:-1], tmp_texts[:-1]):
            dataset_dict["train"].append(
                {
                    'text': text,
                    'simplification': tmp_texts[-1],
                    'title': title
                }
            )
        prev_slug = record['slug']
        prev_title = record['title']
        tmp_texts = []       

In [53]:
print(len(dataset_dict['train']))

5182


#### Test

In [54]:
test_dict = test_df.to_dict(orient='records')

In [55]:
prev_slug = train_dict[0]['slug']
tmp_texts = []
tmp_titles = []

for record in test_dict:
    # Iterate through all the records belonging to a slug and save the texts
    if record['slug'] == prev_slug:
        tmp_texts.append(record['text'])
        tmp_titles.append(record['title'])
    # We've reached a new slug, so it's time to create the train/test record for the given slug
    else:
        for title, text in zip(tmp_titles[:-1], tmp_texts[:-1]):
            dataset_dict["test"].append(
                {
                    'text': text,
                    'simplification': tmp_texts[-1],
                    'title': title
                }
            )
        prev_slug = record['slug']
        prev_title = record['title']
        tmp_texts = []       

In [56]:
print(len(dataset_dict['test']))

1292


In [57]:
prefix = "simplify: "
inputs = []
targets = []
for doc in dataset_dict['train']:
    inputs.append(prefix + doc['text'])
    targets.append(doc['simplification'])

In [58]:
print(len(inputs), len(targets))

5182 5182


In [24]:
prefix = "simplify: "
test_inputs = []
test_targets = []
for doc in dataset_dict['test']:
    test_inputs.append(prefix + doc['text'])
    test_targets.append(doc['simplification'])

In [25]:
print(len(test_inputs), len(test_targets))

1292 1292


### Tokenizer

In [39]:
# Tokenization
# tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

### Preprocessing

In [27]:
def preprocess_function(inputs, targets):
    # inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, return_tensors='pt', max_length=1024, padding='max_length', truncation=True)

    labels = tokenizer(text_target=targets, return_tensors='pt', max_length=1024, padding='max_length', truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs, labels

In [28]:
model_inputs, model_labels = preprocess_function(inputs, targets)

In [29]:
model_inputs['input_ids'][:1]

tensor([[20375,    60,  9874,  ...,   461,     4,     3]])

In [30]:
def shift_tokens_right(input_ids, pad_token_id):
    """Shift input ids one token to the right, and wrap the last non pad token (usually <eos>)."""
    prev_output_tokens = input_ids.clone()
    index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1) - 1).unsqueeze(-1)
    prev_output_tokens[:, 0] = input_ids.gather(1, index_of_eos).squeeze()
    prev_output_tokens[:, 1:] = input_ids[:, :-1]
    return prev_output_tokens

### Train DataLoader

In [27]:
# Create a TensorDataset
dataset = TensorDataset(model_inputs['input_ids'], model_labels['input_ids'])

# Create a DataLoader
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

### Model Training

In [28]:
model = XLNetLMHeadModel.from_pretrained('xlnet-base-cased')
model.load_state_dict(torch.load(f'xlmnet_bs4_max_length1024_4e.pth')['model_state_dict'])
model = model.to(device)

In [29]:
# 4. Define Training Parameters
optimizer = AdamW(model.parameters(), lr=2e-5)

/mnt/c/Users/Sergiu/Desktop/DLComp/venv/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
# Move model to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [31]:
min_loss = 3
for epoch in range(20):  # Number of epochs
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_loader):
        # Assuming batch is a tuple of (input_ids, target_ids)
        input_ids, target_ids = batch
        input_ids, target_ids = input_ids.to(device), target_ids.to(device)
        
        # Shift the target IDs for the decoder input and ignore the last token
        decoder_input_ids = shift_tokens_right(target_ids, tokenizer.pad_token_id)

        # Clear previous gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(input_ids=input_ids, labels=decoder_input_ids)
        loss = outputs.loss

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if total_loss < min_loss:
            min_loss = total_loss

            checkpoint = {
               'model_state_dict': model.state_dict(),
            }
            torch.save(checkpoint, f'xlmnet_bs4_max_length_1024_epoch_{epoch}.pth')
        
        if step % 100 == 0 and not step == 0:
            print(f'Epoch {epoch} Step {step} Loss {loss.item()}')

          

    # Calculate the average loss over the entire batch
    avg_loss = total_loss / len(train_loader)
    print(f"Average Training Loss: {avg_loss}")

Epoch 0 Step 100 Loss 2.9248955249786377
Epoch 0 Step 200 Loss 3.905564069747925
Epoch 0 Step 300 Loss 4.352330207824707
Epoch 0 Step 400 Loss 3.610839366912842
Epoch 0 Step 500 Loss 3.7187016010284424
Epoch 0 Step 600 Loss 3.744738817214966
Epoch 0 Step 700 Loss 4.7788238525390625
Epoch 0 Step 800 Loss 3.6326682567596436
Epoch 0 Step 900 Loss 4.585256576538086
Epoch 0 Step 1000 Loss 5.124723434448242
Epoch 0 Step 1100 Loss 3.506408452987671
Epoch 0 Step 1200 Loss 3.986222743988037
Average Training Loss: 4.112069247129523
Epoch 1 Step 100 Loss 5.028563022613525
Epoch 1 Step 200 Loss 4.4681596755981445
Epoch 1 Step 300 Loss 4.1537580490112305
Epoch 1 Step 400 Loss 4.318265438079834
Epoch 1 Step 500 Loss 4.0916666984558105
Epoch 1 Step 600 Loss 4.466844081878662
Epoch 1 Step 700 Loss 3.6207094192504883
Epoch 1 Step 800 Loss 4.858518123626709
Epoch 1 Step 900 Loss 4.145331859588623
Epoch 1 Step 1000 Loss 4.293227195739746
Epoch 1 Step 1100 Loss 3.7182812690734863
Epoch 1 Step 1200 Loss 3.


KeyboardInterrupt



```
Epoch 0 Step 100 Loss 5.026244163513184
Epoch 0 Step 200 Loss 5.893657684326172
Epoch 0 Step 300 Loss 4.964169979095459
Epoch 0 Step 400 Loss 5.1959733963012695
Epoch 0 Step 500 Loss 4.505870342254639
Epoch 0 Step 600 Loss 4.594396591186523
Epoch 0 Step 700 Loss 3.460613965988159
Epoch 0 Step 800 Loss 4.547153472900391
Epoch 0 Step 900 Loss 4.283635139465332
Epoch 0 Step 1000 Loss 3.9535672664642334
Epoch 0 Step 1100 Loss 4.520374298095703
Epoch 0 Step 1200 Loss 6.07244348526001
Average Training Loss: 4.940993779235416
Epoch 1 Step 100 Loss 5.125080585479736
Epoch 1 Step 200 Loss 3.9568779468536377
Epoch 1 Step 300 Loss 4.50228214263916
Epoch 1 Step 400 Loss 5.492114543914795
Epoch 1 Step 500 Loss 4.65526008605957
Epoch 1 Step 600 Loss 5.31953763961792
Epoch 1 Step 700 Loss 3.8569183349609375
Epoch 1 Step 800 Loss 4.194915294647217
Epoch 1 Step 900 Loss 4.093446254730225
Epoch 1 Step 1000 Loss 3.662853479385376
Epoch 1 Step 1100 Loss 4.537163734436035
Epoch 1 Step 1200 Loss 4.524852752685547
Average Training Loss: 4.4248728595766025
Epoch 2 Step 100 Loss 4.603614330291748
Epoch 2 Step 200 Loss 4.217392921447754
Epoch 2 Step 300 Loss 4.435378074645996
Epoch 2 Step 400 Loss 4.7331438064575195
Epoch 2 Step 500 Loss 4.359727382659912
Epoch 2 Step 600 Loss 5.301181793212891
Epoch 2 Step 700 Loss 4.332385063171387
Epoch 2 Step 800 Loss 3.7514963150024414
Epoch 2 Step 900 Loss 4.650548934936523
Epoch 2 Step 1000 Loss 4.121835231781006
Epoch 2 Step 1100 Loss 3.826198101043701
Epoch 2 Step 1200 Loss 5.222508907318115
Average Training Loss: 4.263375194536315
Epoch 3 Step 100 Loss 4.717140197753906
Epoch 3 Step 200 Loss 4.590120315551758
Epoch 3 Step 300 Loss 4.780978202819824
Epoch 3 Step 400 Loss 4.299923896789551
Epoch 3 Step 500 Loss 4.794330596923828
Epoch 3 Step 600 Loss 4.90772008895874
Epoch 3 Step 700 Loss 5.2156596183776855
Epoch 3 Step 800 Loss 4.793715953826904
Epoch 3 Step 900 Loss 3.985318660736084
Epoch 3 Step 1000 Loss 3.7246477603912354
Epoch 3 Step 1100 Loss 3.602402687072754
Epoch 3 Step 1200 Loss 4.329013347625732
Average Training Loss: 4.173481227071197
```

In [31]:
checkpoint = {
   'model_state_dict': model.state_dict(),
}
torch.save(checkpoint, f'xlmnet_bs4_max_length1024_4e.pth')

## Inference

In [31]:
model = XLNetLMHeadModel.from_pretrained('xlnet-base-cased')
model.load_state_dict(torch.load(f'xlmnet_bs4_max_length1024_4e.pth')['model_state_dict'])
model = model.to(device)
model.eval()

XLNetLMHeadModel(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (l

In [32]:
model_test_inputs, model_test_labels = preprocess_function(test_inputs, test_targets)

In [33]:
# Create a TensorDataset
test_dataset = TensorDataset(model_test_inputs['input_ids'], model_test_labels['input_ids'])

# Create a DataLoader
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

### Generate Rouge Scores

In [34]:
from datasets import load_metric

def evaluate_rouge(model, val_loader, tokenizer, device):
    rouge = load_metric('rouge', trust_remote_code=True)
    model.eval()
    for batch in val_loader:
        input_ids, target_ids = batch
        input_ids, target_ids = input_ids.to(device), target_ids.to(device)
        
        # Generate summaries
        outputs = model.generate(input_ids=input_ids)
        
        # Decode the summaries
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(target_ids, skip_special_tokens=True)

        # Compute Rouge score
        rouge.add_batch(predictions=decoded_preds, references=decoded_labels)
    
    # Calculate final score
    result = rouge.compute()
    for key in result.keys():
        print(f"{key}: {result[key].mid}")

    return result


In [ ]:
%%time
rouge_score = evaluate_rouge(model, test_loader, tokenizer, device)

/tmp/ipykernel_1859/3823000111.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric('rouge', trust_remote_code=True)
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (-1). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


ROUGE Score computed on last 20% of the NEWSELA dataset.
```py
rouge1: Score(precision=0.5682628637119129, recall=0.7606626781061232, fmeasure=0.6417841616651105)
rouge2: Score(precision=0.3472802895392726, recall=0.4498331452513754, fmeasure=0.3873985543099933)
rougeL: Score(precision=0.3939804996636087, recall=0.5136579508637882, fmeasure=0.4403192447961841)
rougeLsum: Score(precision=0.3938001014396969, recall=0.5137175834759207, fmeasure=0.440159675368905
```16)

### Generate Simplification

In [35]:
def generate_simplification(text, model, tokenizer, device):
    model.eval()
    inputs = tokenizer(text, return_tensors='pt', max_length=1024, padding='max_length', truncation=True).to(device)
    
    # Generate summary
    simplification_ids = model.generate(inputs['input_ids'], min_length=40, max_length=1025, length_penalty=2.0, num_beams=4, early_stopping=True)
    simplification = tokenizer.decode(simplification_ids[0], skip_special_tokens=True)
    
    return simplification

In [36]:
prompt = """
Coronaviruses are a group of related RNA viruses that cause diseases in mammals and birds. In humans and birds, they cause respiratory tract infections that can range from mild to lethal. Mild illnesses in humans include some cases of the common cold (which is also caused by other viruses, predominantly rhinoviruses), while more lethal varieties can cause SARS, MERS and COVID-19, which is causing the ongoing pandemic. In cows and pigs they cause diarrhea, while in mice they cause hepatitis and encephalomyelitis.

Coronaviruses constitute the subfamily Orthocoronavirinae, in the family Coronaviridae, order Nidovirales and realm Riboviria.[3][4] They are enveloped viruses with a positive-sense single-stranded RNA genome and a nucleocapsid of helical symmetry.[5] The genome size of coronaviruses ranges from approximately 26 to 32 kilobases, one of the largest among RNA viruses.[6] They have characteristic club-shaped spikes that project from their surface, which in electron micrographs create an image reminiscent of the stellar corona, from which their name derives.[7]

Etymology
The name "coronavirus" is derived from Latin corona, meaning "crown" or "wreath", itself a borrowing from Greek κορώνη korṓnē, "garland, wreath".[8][9] The name was coined by June Almeida and David Tyrrell who first observed and studied human coronaviruses.[10] The word was first used in print in 1968 by an informal group of virologists in the journal Nature to designate the new family of viruses.[7] The name refers to the characteristic appearance of virions (the infective form of the virus) by electron microscopy, which have a fringe of large, bulbous surface projections creating an image reminiscent of the solar corona or halo.[7][10] This morphology is created by the viral spike peplomers, which are proteins on the surface of the virus.[11]

The scientific name Coronavirus was accepted as a genus name by the International Committee for the Nomenclature of Viruses (later renamed International Committee on Taxonomy of Viruses) in 1971.[12] As the number of new species increased, the genus was split into four genera, namely Alphacoronavirus, Betacoronavirus, Deltacoronavirus, and Gammacoronavirus in 2009.[13] The common name coronavirus is used to refer to any member of the subfamily Orthocoronavirinae.[4] As of 2020, 45 species are officially recognised.[14]
"""

In [37]:
simplification = generate_simplification(prompt, model, tokenizer, device)

In [38]:
print(simplification)

Coronaviruses are a group of related RNA viruses that cause diseases in mammals and birds. In humans and birds, they cause respiratory tract infections that can range from mild to lethal. Mild illnesses in humans include some cases of the common cold (which is also caused by other viruses, predominantly rhinoviruses), while more lethal varieties can cause SARS, MERS and COVID-19, which is causing the ongoing pandemic. In cows and pigs they cause diarrhea, while in mice they cause hepatitis and encephalomyelitis. Coronaviruses constitute the subfamily Orthocoronavirinae, in the family Coronaviridae, order Nidovirales and realm Riboviria.[3][4] They are enveloped viruses with a positive-sense single-stranded RNA genome and a nucleocapsid of helical symmetry.[5] The genome size of coronaviruses ranges from approximately 26 to 32 kilobases, one of the largest among RNA viruses.[6] They have characteristic club-shaped spikes that project from their surface, which in electron micrographs cre